# Проект для «Викишоп»

<div class="alert alert-block alert-info">
    
Интернет-магазин «Викишоп» запускает новый сервис: клиентам можно предлагать свои правки и комментировать изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии пользователей, редактирующих и дополняющих описание товаров, и отправлять их на модерацию. 

Цель - обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности правок.

## Подготовка

In [16]:
# загрузка библиотек
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer 
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [17]:
# открытие файла
df = pd.read_csv('/content/toxic_comments.csv')

# функция для изучения данных датафрейма
def open_info(data):
    # вывод 5 строк данных
    print('*******************************************************')
    print('Представление датафрейма')
    display(data.head())
    print('*******************************************************')
    # вывод информации о датафрейме
    print('Общая информация о датафрейме')
    print('')
    print(data.info())
    print('*******************************************************')
    # проверка на наличие пропусков
    print('Пропуски:', data.isna().sum().sum())
    print('*******************************************************')
    # проверка на наличие дубликатов
    print('Дубликаты:', data.duplicated().sum())

In [19]:
open_info(df)

*******************************************************
Представление датафрейма


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


*******************************************************
Общая информация о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None
*******************************************************
Пропуски: 0
*******************************************************
Дубликаты: 0


Итак, в датафрейме 159571 строка и 2 столбца: 'toxic'(обозначения того, является ли текст токсичным или нет), 'text' (тексты твитов). Дубликатов и пропусков нет.  

Посмотрим, сколько у нас токсичных твитов.

In [20]:
# количество токсичных и нетоксичных твитов
display(df['toxic'].value_counts())
# соотношение
df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]


0    143346
1     16225
Name: toxic, dtype: int64

8.834884437596301

Классы не сбалансированы. Видим, что токсичных твитов не так много, если сравнивать с количеством нетоксичных твитов. Нужно при обучении моделей это учесть.

Прежде чем извлечь признаки из текста, нужно его упростить. Для этого необходимо сделать лемматизацию, очищение текста и токенизацию.

In [21]:
# функция, чтобы вернуть слово к начальной форме
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# функция лемматизации
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    text = text.lower()
    lemm_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in lemm_list])      
    return lemm_text

# функция очищения текста
def clear_text(text):
    text = re.sub(r"\'m", " am ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"there's", "there is ", text)
    text = re.sub(r"it's", "it is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    clear_text = re.sub(r'[^a-zA-Z]', ' ', text)
    clear_text = clear_text.lower().split()
    return ' '.join(clear_text)

# токенизация
df['lemm_text'] = df['text'].apply(lambda text: lemmatize(clear_text(text)))

# проверка
display(df.loc[0, 'text'])
df.loc[0, 'lemm_text']

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

'explanation why the edits make under my username hardcore metallica fan be revert they be not vandalism just closure on some gas after i vote at new york doll fac and please do not remove the template from the talk page since i be retire now'

Токенизация прошла успешно. Сохраним полученные данные в новом датафрейме.

In [22]:
# сохранение файла
df.to_csv('lemm_toxic.csv', index=False)

# открытие файла
df = pd.read_csv('lemm_toxic.csv')

Подготовим признаки для обучения моделей.

In [23]:
# признаки и целевой признак
features = df['lemm_text']
target = df['toxic']

# создание тренировочной и тестовой выборок
features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target,
                                                                            test_size = 0.3,
                                                                            random_state=12345)
# проверка
display(features_train.shape)
features_test.shape

(111699,)

(47872,)

In [24]:
# Создададим корпуса слов для обучающей и тестовой выборок:
corpus_train = features_train.values.astype('U')
corpus_test = features_test.values.astype('U')

In [25]:
# очищение текста от стоп-слов и ненужных символов
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [26]:
# получение TF-IDF для корпуса текста
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tfidf_train = count_tf_idf.fit_transform(corpus_train)
tfidf_test = count_tf_idf.transform(corpus_test)
# размеры матриц
display(tfidf_train.shape)
tfidf_test.shape

(111699, 122927)

(47872, 122927)

Таким образом, мы подготовили данные для обучения модели. Преобразовали твиты в векторный вид, необходимый для обучения моделей. TF-IDF увеличил количество признаков, равное количеству в мешке слов.

## Обучение

Обучим несколько моделей и оценим полученные результаты.
Модели для обучения: LogisticRegression и DecisionTreeClassifier.

### Модель LogisticRegression

In [27]:
model_LR = LogisticRegression()
model_LR.fit(tfidf_train, target_train)
predictions = model_LR.predict(tfidf_train)
train_f1 = f1_score(target_train, predictions)

print('F1 на тренировочной выборке: {:.2f}'.format(train_f1))

F1 на тренировочной выборке: 0.76


### Модель DecisionTreeClassifier

In [28]:
# подбор параметров
#best_result = 0
#for depth in range(1, 20):
    #model_DTC = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    #model_DTC.fit(tfidf_train, target_train)
    #predicted = model_DTC.predict(tfidf_train)
    #result = f1_score(target_train, predicted)
    #if result > best_result:
        #best_result = result
        #best_depth = depth
#print('Глубина дерева:', best_depth)
#print('F1-мера:', best_result)

Лучший результат при depth = 19.

In [29]:
model_DTC = DecisionTreeClassifier(max_depth = 19, random_state=12345)
model_DTC.fit(tfidf_train, target_train)
predicted = model_DTC.predict(tfidf_train)
train_f1 = f1_score(target_train, predicted)

print('F1 на тренировочной выборке: {:.2f}'.format(train_f1))

F1 на тренировочной выборке: 0.70


По метрике F1 на тренировочной выборке лучшая модель LogisticRegression. Посмотрим, какова метрика F1 на тестовых выборках.

## Тестирование моделей

Сделаем функцию для тестирования моделей.

In [30]:
# функция скоринга для определения метрики F1 для тестовой выбороки:
def scoring(model):
    test_pred = model.predict(tfidf_test)
    test_f1 = f1_score(target_test, test_pred)
    
    print('F1 на тестовой выборке: {:.2f}'.format(test_f1))

In [31]:
print('LogisticRegression')
scoring(model_LR)
print('DecisionTreeClassifier')
scoring(model_DTC)

LogisticRegression
F1 на тестовой выборке: 0.74
DecisionTreeClassifier
F1 на тестовой выборке: 0.66


F1 выше у модели LogisticRegression.
Поскольку классы неуравновешены, попробуем обучить модель с балансировкой классов.

In [32]:
model_LR_bal = LogisticRegression(class_weight='balanced')
model_LR_bal.fit(tfidf_train, target_train)
predictions = model_LR_bal.predict(tfidf_train)
train_f1 = f1_score(target_train, predictions)

print('F1 на тренировочной выборке: {:.2f}'.format(train_f1))

F1 на тренировочной выборке: 0.83


В этом случае F1 оказывается выше, протестируем модель.

In [33]:
# тестирование модели
print('LogisticRegression')
scoring(model_LR_bal)

LogisticRegression
F1 на тестовой выборке: 0.75


При балансировке метрика F1 оказывается выше.

## Выводы

<div class="alert alert-success">
    
Таким образом, была сделана попытка обучить модель классифицировать комментарии на позитивные и негативные и достигнуто значение 0.75 по метрике F1. Перед обучением модели было сделано упрощение текста, в частности его лемматизация, очищение и токенизация. Далее все твиты были преобразованы в векторный вид, необходимый для обучения моделей. TF-IDF увеличил количество признаков, равное количеству в мешке слов. Для обучения были выбраны две модели: LogisticRegression и DecisionTreeClassifier. Также был проведен эксперимент с обучением модели с изменением весов классов. В результате был сделан вывод, лучше обучается классифицировать комментарии на позитивные и негативные модель LogisticRegression с изменением весов классов.